In [40]:
import pandas as pd
import numpy as np

data = pd.read_csv('./hw1/train.csv', encoding='big5')

In [49]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()


In [42]:
month_data= {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24: (day+1) * 24] = raw_data[18 * (20 * month + day): 18 * (20 * month + day + 1),:]
        
    month_data[month] = sample

x = np.empty([12 * 471, 18 * 9],dtype=float)
y = np.empty([12 * 471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour,:] = month_data[month][:, day * 24 + hour: day * 24 + hour + 9].reshape(1,-1)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9,day * 24 + hour + 9]

In [43]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [44]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8):, :]
y_validation = y[math.floor(len(y) * 0.8):, :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [45]:
dim = 18 * 9 + 1
w = np.ones([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis=1).astype(float)
learning_rate = 100
iter_time = 10000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2)) / 471 / 12)
    if (t % 100 == 0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)

0:51.65880600054525
100:20.77437564222039
200:14.127460058537881
300:11.30385767078897
400:9.757354637574426
500:8.790692587072341
600:8.132230172649814
700:7.657473854962996
800:7.30155272831573
900:7.027160886807601
1000:6.81117773487349
1100:6.638413442067583
1200:6.4984178315454
1300:6.383737204109007
1400:6.288904904313377
1500:6.209824896717986
1600:6.143376719550693
1700:6.087151427316504
1800:6.039268829370359
1900:5.998247451793656
2000:5.962910016891086
2100:5.932313603545214
2200:5.905697380038531
2300:5.882443083695772
2400:5.862044880037761
2500:5.844086201253044
2600:5.828221825181654
2700:5.8141639193771475
2800:5.801671105507553
2900:5.790539838697473
3000:5.780597571519529
3100:5.771697301555401
3200:5.76371319747139
3300:5.7565370703528185
3400:5.750075511024628
3500:5.744247554882434
3600:5.738982766741748
3700:5.7342196618558665
3800:5.729904397378539
3900:5.72598968250727
4000:5.722433866343132
4100:5.719200170896092
4200:5.716256043216149
4300:5.713572605766973
44

In [50]:
w = np.load('weight.npy')
x_validation = np.concatenate((np.ones([1131, 1]), x_validation), axis=1).astype(float)
ans_y = np.dot(x_validation, w)
loss = np.sqrt(np.sum(np.power(ans_y - y_validation, 2)) / 1131)
print(loss)

ValueError: shapes (1131,164) and (163,1) not aligned: 164 (dim 1) != 163 (dim 0)

In [81]:
testdata = pd.read_csv('./hw1/test.csv', header=None, encoding='big5')

In [82]:
test_data = testdata.iloc[:, 2:]

In [83]:
test_data[test_data == 'NR'] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [84]:
test_data = test_data.to_numpy()

In [85]:
test_x = np.empty([240, 18 * 9], dtype=float)
for i in range(240): 
    test_x[i, :] = test_data[18 * i: 18 * (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis=1).astype(float) 

In [86]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)

In [89]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 6.675990980112434]
['id_1', 18.071674932253515]
['id_2', 24.038225880811623]
['id_3', 7.7722579903982485]
['id_4', 27.02666876933243]
['id_5', 22.285593028075088]
['id_6', 23.89889844954017]
['id_7', 30.419205101639704]
['id_8', 16.862185194023503]
['id_9', 59.6070626547926]
['id_10', 11.72114579899808]
['id_11', 9.268966163376234]
['id_12', 63.27717290776698]
['id_13', 53.28392517194696]
['id_14', 22.196853423304336]
['id_15', 12.170905086948896]
['id_16', 32.541408284225334]
['id_17', 66.68114491778043]
['id_18', -0.4288923754744829]
['id_19', 17.081744200799683]
['id_20', 41.78232474057129]
['id_21', 72.12298475750185]
['id_22', 9.288776353342502]
['id_23', 17.798397558856912]
['id_24', 14.71726572618007]
['id_25', 37.710786473470165]
['id_26', 14.56856476463511]
['id_27', 69.34780229867519]
['id_28', 7.269607376635076]
['id_29', 55.42394068387628]
['id_30', 24.4647107388274]
['id_31', 8.490068429392878]
['id_32', 2.485893898490218]
['id_33', 18.619693900032